In [1]:
# Initial imports
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math

%matplotlib inline

/Users/Apple/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import cv2
import numpy as np

In [3]:
def initialiseBlobDetector():
    params = cv2.SimpleBlobDetector_Params()
    # Change thresholds
    params.blobColor = 0
    # Create a detector with the parameters
    ver = (cv2.__version__).split('.')
    if int(ver[0]) < 3 :
        detector = cv2.SimpleBlobDetector(params)
    else : 
        detector = cv2.SimpleBlobDetector_create(params)
    return detector

In [4]:
def smothenImage(im):
    kernel = np.ones((5,5),np.float32)/22
    dst = cv2.filter2D(im,-1,kernel)
    return dst

In [5]:
def showImages(im, im2, im3):
    cv2.imshow("Original", im)
    cv2.imshow("With Detection",im2);
    cv2.imshow("Generated",im3);
    cv2.waitKey(0)
    cv2.imwrite("Original.BMP", im)
    cv2.imwrite("WithDetection.BMP", im2)
    cv2.imwrite("Generated.BMP", im3)

In [6]:
def showDetection(im, keypoints, keypointsData):
    im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    print "Number of Blobs : " + str(len(keypoints))
    # Show keypoints
    im2 = createImageCopy(im, keypointsData)
    showImages(im, im_with_keypoints, im2)

In [7]:
def extractKeypointsData(keypoints):
    data = DataFrame(columns=('PtX', 'PtY', 'Diameter'))
    for keypoint in keypoints:
        PtX = keypoint.pt[0]
        PtY = keypoint.pt[1]
        Area = keypoint.size
        keyData = DataFrame([[PtX, PtY, Area]], columns=('PtX', 'PtY', 'Diameter'))
        data = data.append(keyData, ignore_index = True)
        data['Radius'] = data['Diameter']/2
    return data

In [8]:
def createImageCopy(im, data):
    im2 = im.copy()
    height, width = im.shape
    cv2.rectangle(im2,(0,0),(width,height),(80,80,81),-1)
    for i in range(0, data.shape[0]):
        X = data.iloc[[i]]['PtX']
        Y = data.iloc[[i]]['PtY']
        Radius = data.iloc[[i]]['Radius']
        cv2.circle(im2,(X,Y), Radius, (0,0,0), -1)
    return im2

In [25]:
im = cv2.imread("1002.BMP", cv2.IMREAD_GRAYSCALE)
im = smothenImage(im)
detector = initialiseBlobDetector()

keypoints = detector.detect(im)
keypointsData = extractKeypointsData(keypoints)
# showDetection(im, keypoints, keypointsData)
# imFinal = createImageCopy(im, keypointsData)

In [27]:
height, width = im.shape
area = height*width
keypointsData['AreaOfBlobs'] = np.pi*keypointsData['Radius']**2
volumeFraction = keypointsData['AreaOfBlobs'].sum() / area
print 'Volume Fraction of the input Composite image is ' + str(volumeFraction)

Volume Fraction of the input Composite image is 0.152864484281


In [28]:
keypointsData

,Diameter,PtX,PtY,Radius,AreaOfBlobs
0,12.836687,63.091553,269.027771,6.418344,129.418330
1,20.303453,78.846146,263.404114,10.151727,323.764859
2,10.625246,284.939880,247.163132,5.312623,88.668196
3,15.425774,340.525360,241.271255,7.712887,186.889021
4,15.390263,341.608276,183.140594,7.695131,186.029553
5,13.618959,320.703003,155.688995,6.809480,145.672554
6,13.585695,8.895333,151.381897,6.792848,144.961815
7,19.387796,54.891918,136.842850,9.693898,295.220684
8,11.156711,305.667511,105.503250,5.578355,97.760231
9,9.219251,32.816753,99.672607,4.609625,66.754589
